In [2]:
import os
import sys
import time
import json
import shutil
import warnings
warnings.filterwarnings('ignore')
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from metrics import *
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from datasets import *
from utils import *
from models import *

# Finals

In [175]:
def show_results(model_name, k=10, mode=1):
    results = []
    for i in range(3):
        res = pickle.load(open("/home/user/wanzhang/Trans4Ab/Transformer4Ab-locla/results/CoV-AbDab/{}/result_{}.pkl".format(model_name, i), "rb"))
        results.append(res)
    #     print(res)

    acc = []
    f1 = []
    auc = []
    gmean = []
    mcc = []
    for i in range(3):
        acc.extend(results[i][1])
        f1.extend(results[i][2])
        auc.extend(results[i][3])
        gmean.extend(results[i][4])
        mcc.extend(results[i][5])

    # average all 30 results for mean, and regard 30 as whole for std
    if mode==0:
        print(model_name.split("_")[0], "\n")
        print("acc:\t{:.4f} ± {:.4f}".format(np.mean(acc), np.std(acc)))
        print("f1:\t{:.4f} ± {:.4f}".format(np.mean(f1), np.std(f1)))
        print("auc:\t{:.4f} ± {:.4f}".format(np.mean(auc), np.std(auc)))
        print("gmean:\t{:.4f} ± {:.4f}".format(np.mean(gmean), np.std(gmean)))
        print("mcc:\t{:.4f} ± {:.4f}".format(np.mean(mcc), np.std(mcc)))

    # calculate mean and std for the same fold in three times, then average the mean and std for 10 folds
    elif mode==1:
        accs = []
        f1s = []
        aucs = []
        gmeans = []
        mccs = []
        for i in range(k):
            accs.append(np.std([acc[i], acc[i+k], acc[i+k+k]]))
            f1s.append(np.std([f1[i], f1[i+k], f1[i+k+k]]))
            aucs.append(np.std([auc[i], auc[i+k], auc[i+k+k]]))
            gmeans.append(np.std([gmean[i], gmean[i+k], gmean[i+k+k]]))
            mccs.append(np.std([mcc[i], mcc[i+k], mcc[i+k+k]]))


        print(model_name.split("_")[0], "\n")
        print("acc:\t{:.4f} ± {:.4f}".format(np.mean(acc), np.mean(accs)))
        print("f1:\t{:.4f} ± {:.4f}".format(np.mean(f1), np.mean(f1s)))
        print("auc:\t{:.4f} ± {:.4f}".format(np.mean(auc), np.mean(aucs)))
        print("gmean:\t{:.4f} ± {:.4f}".format(np.mean(gmean), np.mean(gmeans)))
        print("mcc:\t{:.4f} ± {:.4f}".format(np.mean(mcc), np.mean(mccs)))

    # calculate mean and std for 1/2/3 times of 10fold, then mean and std the three results
    elif mode==2:
        accs = []
        f1s = []
        aucs = []
        gmeans = []
        mccs = []
        for i in range(3):
            accs.append(np.std([acc[i*k:i*k+k]]))
            f1s.append(np.std([f1[i*k:i*k+k]]))
            aucs.append(np.std([auc[i*k:i*k+k]]))
            gmeans.append(np.std([gmean[i*k:i*k+k]]))
            mccs.append(np.std([mcc[i*k:i*k+k]]))
            
        print(model_name.split("_")[0], "\n")
        print("acc:\t{:.4f} ± {:.4f}".format(np.mean(acc), np.mean(accs)))
        print("f1:\t{:.4f} ± {:.4f}".format(np.mean(f1), np.mean(f1s)))
        print("auc:\t{:.4f} ± {:.4f}".format(np.mean(auc), np.mean(aucs)))
        print("gmean:\t{:.4f} ± {:.4f}".format(np.mean(gmean), np.mean(gmeans)))
        print("mcc:\t{:.4f} ± {:.4f}".format(np.mean(mcc), np.mean(mccs)))
    
    else:
        print("wrong mode!")
        exit()

In [136]:
show_results(model_name = "lstm_07-14-22-21-42")

lstm 

acc:	0.7301 ± 0.0106
f1:	0.8382 ± 0.0080
auc:	0.5054 ± 0.0565
gmean:	0.0294 ± 0.0416
mcc:	-0.0015 ± 0.0347
